In [147]:
import pattern
from pattern.web import URL, DOM, plaintext

from os import listdir
from os.path import isfile, join
import os

import pandas as pd
import numpy as np
import nltk
from collections import Counter, defaultdict
from nltk.corpus import stopwords
import string
import random

filedic = {}
for each in os.listdir(os.getcwd())[2:]:
    if (each != "lyrics.ipynb") and (each != 'Untitled.ipynb'):
        filedic[each] = listdir(each)[2:]

In [156]:
lyricsdic = defaultdict(list)
exclude = set(string.punctuation)

remove_list = ['verse','hook','chorus', '','choruslloyd banks', '50 cent', 'outro','bridge', 'chorus', 'lyrics', 'lloyd banks', 'rick ross','lil wayne', 'da', 'dem', 'im', 'ya', 'dat', 'tony yayo', 'banks yay 50',
              'verse 1 tony yayo', 'verse 4 lloyd banks', 'verse 5 50 cent', 'verse 2 tony yayo', 'verse 1 50 cent', 'chorus 50 cent', 'intro 50 cent', 'biggie sample  in the background throughout the song', 'verse 3  tony yayo', 'verse 1  50 cent','chorus  50 cent','verse 2  lloyd banks','verse 2 lloyd banks', 'verse 6 tony yayo', 'verse 4 yony yayo']

for k,v in filedic.iteritems():
    for link in v:
        page = open(k+'/'+link).read()
        dom = DOM(page)
        lyrics = dom('.lyrics')[0]
        p = plaintext(lyrics.content)
        p = ''.join(ch for ch in p if ch not in exclude)
        p = p.splitlines()
        lyricsdic[k].append(filter(lambda x: x.lower() not in remove_list, p))

In [163]:
#lyricsdic['50cent'][99]

In [126]:
lineslist = []
remove_list = ['verse','hook','chorus', '','choruslloyd banks', '50 cent', 'outro','bridge', 'chorus', 'lyrics', 'lloyd banks', 'rick ross','lil wayne', 'da', 'dem', 'im', 'ya', 'dat', 'tony yayo', 'banks yay 50',
              'verse 1 tony yayo', 'verse 4 lloyd banks', 'verse 5 50 cent', 'verse 2 tony yayo', 'verse 1 50 cent', 'chorus 50 cent', 'intro 50 cent', 'biggie sample  in the background throughout the song', 'verse 3  tony yayo', 'verse 1  50 cent','chorus  50 cent','verse 2  lloyd banks','verse 2 lloyd banks', 'verse 6 tony yayo', 'verse 4 yony yayo']
exportlist = []
for each in lyricslist:
    lineslist.append(each.splitlines())
lineslist = np.array(lineslist)
for each in lineslist:
    exportlist.append(filter(lambda x: x.lower() not in remove_list, each))

In [161]:
import pickle
f = open('picklelyrics', 'wb')

pickle.dump(lyricsdic, f)

# import simplejson
# f = open('lyricsarchive.txt', 'w')
# simplejson.dump(exportlist, f)
# f.close()

In [133]:
tokenlist = []

for lyrics in lyricslist:
    tokens = nltk.word_tokenize(lyrics)
    tokenlist.extend([w for w in tokens if w.lower() not in stopwords.words('english')])

import random

class Markov(object):

    def __init__(self, words):
        self.cache = {}
        #self.open_file = open_file
        self.words = words
        self.word_size = len(self.words)
        self.database()

    def file_to_words(self):
        self.open_file.seek(0)
        data = self.open_file.read()
        words = data.split()
        return words

    def triples(self):
        if len(self.words) < 3:
            return
        for i in range(len(self.words) - 2):
            yield (self.words[i], self.words[i+1], self.words[i+2])

    def database(self):
        for w1, w2, w3 in self.triples():
            key = (w1, w2)
            if key in self.cache:
                self.cache[key].append(w3)
            else:
                self.cache[key] = [w3]

    def generate_markov_text(self, size=25):
        seed = random.randint(0, self.word_size-3)
        seed_word, next_word = self.words[seed], self.words[seed+1]
        w1, w2 = seed_word, next_word
        gen_words = []
        for i in xrange(size):
            gen_words.append(w1)
            w1, w2 = w2, random.choice(self.cache[(w1, w2)])
        gen_words.append(w2)
        return ' '.join(gen_words)

In [162]:
mark = Markov(tokenlist)
mark.generate_markov_text(size = 50)

u'hesitate pull right back Tell something good baby Boss done took Im back brother went Thats tomb say Right government Dumile Either unmarked engraved hey whos say Pass mic like yin yang Clang Crime dont pay ya taxes Fore spit name backwards Aaask around Im back nigga hold uh SmithNWesson Im'

In [135]:

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from collections import Counter
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import linkage, dendrogram

vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(lyricslist)
features = vectorizer.get_feature_names()
kmeans = KMeans()
kmeans.fit(X)

top_centroids = kmeans.cluster_centers_.argsort()[:,-1:-6:-1]
print "top features for each cluster:"
for num, centroid in enumerate(top_centroids):
    print "%d: %s" % (num, ", ".join(features[i] for i in centroid))

top features for each cluster:
0: lyrics, song, help, visitors, submit
1: 50, cent, im, nigga, niggas
2: like, im, got, know, tha
3: nigga, niggas, im, got, like
4: girl, like, baby, im, love
5: im, like, bitch, nigga, got
6: im, like, got, dont, money
7: da, im, ya, dat, like


In [41]:
def rhyme(inp, level):
    entries = nltk.corpus.cmudict.entries()
    syllables = [(word, syl) for word, syl in entries if word == inp]
    rhymes = []
    for (word, syllable) in syllables:
        rhymes += [word for word, pron in entries if pron[-level:] == syllable[-level:]]
    return set(rhymes)


{u'aglo',
 u'aglow',
 u'belleau',
 u'below',
 u'bleau',
 u'blow',
 u'blowe',
 u'boileau',
 u'celo',
 u'cloe',
 u'clow',
 u'enloe',
 u'enlow',
 u'enslow',
 u'flo',
 u'floe',
 u'flow',
 u'flowe',
 u'furlaud',
 u'glo',
 u'gloe',
 u'glow',
 u'groleau',
 u'guillot',
 u'hello',
 u'inlow',
 u'ledlow',
 u'leleux',
 u'lo',
 u'loe',
 u'loew',
 u'loewe',
 u'loh',
 u'low',
 u'lowe',
 u'mealo',
 u'overflow',
 u'plough',
 u'rouleau',
 u'sloe',
 u'slow',
 u'soileau',
 u'tableau',
 u'tableaux',
 u'tourtelot',
 u'valleau',
 u'veilleux'}